## GraphQL sample sender

Send all possible graphql requests defined in SDL throw proxy. Helps in security testing graphql applications.

Input is output of `gql-generator`, witch parse SDL.

https://github.com/timqian/gql-generator

```
npm install gql-generator -g
gqlg --schemaFilePath ./example/sampleTypeDef.graphql --destDirPath ./example/output --depthLimit 2
```
`depthLimit` should equals 2

You have to specify  
`files` - list of gpl files from `gqlg` with requests  
`target_uri` - target uri  
`http_proxy` if you want to catch requests in Burp  
`dafault_table` is default values for common types.  

Script sends multiple HTTP requests for one request in file if request takes parameters with type with multiple default values.

For example 
`query1(var1: $var1, var2: $var2){..`

if var1 and var2 have type ID!, than script sends 4 request:
```
var1: "1", var2: "1"
var1: "1", var2: "9ed466cc-c371-11e6-934f-1579baf24309"
var1: "9ed466cc-c371-11e6-934f-1579baf24309", var2: "1"
var1: "9ed466cc-c371-11e6-934f-1579baf24309", var2: "9ed466cc-c371-11e6-934f-1579baf24309"
```

If parameter is nullable script exclude it.

In [1]:
import re
import requests
import json
import urllib3
urllib3.disable_warnings()


In [416]:
files = ['businessAdmin.gql',
'externalContent.gql',
'gamification.gql',
'student.gql',
'systemAdmin.gql',
'teacher.gql',
'user.gql'
        ]

target_uri = 'https://x/graphql?access_token=%s'
access_token = 'xxx'
target_uri = target_uri % access_token
http_proxy   = "http://localhost:8080"

proxyDict = { 
             "https"   : http_proxy
            }

In [389]:
def run_query(query): # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post(target_uri, json=query, proxies=proxyDict,verify=False)#headers=headers
    if request.status_code == 200:
        return request.json()
    else:
        return request
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, request.text))


In [407]:

default_table = {'String':['"Hahaha"'],
                 'ID':['"1"','"9ed466cc-c371-11e6-934f-1579baf24309"'],
                 '[ID!]':[['"1"'],['"9ed466cc-c371-11e6-934f-1579baf24309"']],
                 'Int':['"1"']}


In [391]:
def apply_re_sub_for_strings(strings,pattern,replaces):
    res = []
    for s in strings:
        for r in replaces:
            res.append(re.sub(pattern,r,s))
    return res

In [417]:
for fl in files:
    data = open(fl).read()
    l1 = data.split('\n')[0]
    variables_s = re.findall('(\$[^:]*):\s([^,\)]*)',l1)
    var_types = {}
    for x in variables_s:
        var_types[x[0]] = x[1]
    request_type = re.findall('^[^\ ]*',data)[0]
    module = re.findall('\n\ {4}([^\ \n]*)\{',data)[0]
    functions = re.findall('(\ {8}[^\n]*\{[^\}]*})',data)
    new_query_tmpl = 'query{\n' + data.split('\n')[1] + '''
%s    
    }
}'''
    for f in functions:
        #if 'getTaskAvailableSchools' not in f:
        #    continue
        function_vars = re.findall('(\$[^,\)]*)',f)
        cur_funcs = [f]
        try:
            for v in function_vars:
                t = var_types[v]
                if t[-1]!='!':#null possible
                    cur_funcs = apply_re_sub_for_strings(cur_funcs,
                                                         ',?\\s?[^\(,]*\\%s(,?\)?)'%v,
                                                        ['\\1'])
                    #print cur_funcs
                    #cur_func = re.sub(',?\\s?[^\(,]*\\%s,?([\)]?)'%v,'\\1',cur_func)
                    continue
                t = t[:-1]
                if t not in default_table:
                    raise Exception('No default value for type: %s' % t)
                else:#paste default value
                    cur_funcs = apply_re_sub_for_strings(cur_funcs,
                                        '(,?\\s?[^\(,]*)\\%s(,?\)?)'%v
                                        ,['\\1%s\\2'%x for x in default_table[t]])
                    #print cur_funcs
                    #re.sub('(,?\\s?[^\(,]*)\\%s,?([\)]?)'%v,'\\1%s\\2'%default_table[t],cur_func)
            cur_funcs = apply_re_sub_for_strings(cur_funcs,
                                                '(\(\))',[''])
           
            for cur_func in cur_funcs:
                json_request = {'operationName':"",
                   'variables':{},
                   'query':new_query_tmpl%(cur_func)}
                run_query(json_request)
        except Exception as error:
            print str(error)
            continue


No default value for type: ContentProviderInput
No default value for type: ContentLoadInput
No default value for type: FilterExternalTaskInput
No default value for type: DateTime
No default value for type: DateTime
No default value for type: DateTime
No default value for type: DateTime
